In [54]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,r2_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.set_option('float_format', '{:f}'.format)

In [55]:
# feature_cols = user_merged.columns.drop(['business_id', 'name', 'address', 'city', 'state_left', 'postal_code',
# #                                     'census_tract', 'median_income', 'review_id', 'review_count_y', 'user__average_stars',
#                                         'user_id', 'review_stars', 'text', 'date', 'review_count_y', 'user__average_stars',
#                                         'rating_score','income_range'])

In [56]:
yelp_compiled = pd.read_csv('Yelp_data/yelp_compiled.csv')


In [57]:
#stripping white space in front of words in column and getting dummy
yelp_compiled["new_categories"] = yelp_compiled['new_categories'].str.lstrip()
yelp_compiled_cat = yelp_compiled['new_categories'].str.get_dummies(',')

In [58]:
yelp_compiled

,business_id,name,address,city,state_left,postal_code,latitude,longitude,stars,review_count,new_categories,price_range,romantic,intimate,touristy,hipster,divey,classy,trendy,upscale,casual,category,geometry,geoid,county,census_tract,pop_2018,median_income
0,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,2450 E Indian School Rd,Phoenix,AZ,85016.000000,33.495194,-112.028588,3.000000,18,Fast Food,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,POINT (-112.0285876 33.4951941),4013108400,Maricopa County,1084.000000,5195,53377.000000
1,2DUdvdQE8FOxCHHp59HHzw,Bertha's Café,3134 E Indian School Rd,Phoenix,AZ,85016.000000,33.495821,-112.014668,4.500000,381,"Cafes, Desserts, Sandwiches, Breakfast & Brunch, American, Food, Bakeries",1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0,POINT (-112.0146683135 33.4958205544),4013108400,Maricopa County,1084.000000,5195,53377.000000
2,MM8n7SPaIuMlDGbUM5SZhA,AMC Dine-in Theatres Esplanade 14,2515 E Camelback Rd,Phoenix,AZ,85016.000000,33.508571,-112.027642,3.000000,763,"Arts & Entertainment, Cinema, Beer, Wine & Spirits, American, Food",2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0,POINT (-112.027642 33.508571),4013108400,Maricopa County,1084.000000,5195,53377.000000
3,dPFGRDC3Hsi1mDKxvKz_9Q,MercBar,2525 E Camelback Rd,Phoenix,AZ,85016.000000,33.508730,-112.027564,4.000000,218,"Wine Bars, Cocktail Bars, American, Bars, Nightlife, Lounges",2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0,POINT (-112.027564 33.50873),4013108400,Maricopa County,1084.000000,5195,53377.000000
4,iGvnkEz2efD9q-Z7ZcasWQ,Papa John's Pizza,2836 E Indian School Rd,Phoenix,AZ,85016.000000,33.495436,-112.019786,1.500000,11,"Italian, Pizza",1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,POINT (-112.0197856 33.4954365),4013108400,Maricopa County,1084.000000,5195,53377.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6053,E8vu8qm6r5NuuO_Cw-jzyQ,McDonald's,29555 N Cave Creek Rd,Phoenix,AZ,85331.000000,33.753671,-111.990915,1.500000,25,Fast Food,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,POINT (-111.9909153879 33.7536713411),4013612800,Maricopa County,6128.000000,2211,100298.000000
6054,h9pEt23IAyjBtUaYA9v46A,La Poblanita,4012 N 75th Ave,Phoenix,AZ,85033.000000,33.493326,-112.221016,4.500000,115,Mexican,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0,POINT (-112.2210158 33.4933261),4013109704,Maricopa County,1097.040000,2783,44526.000000
6055,Qb1tuREKTsyVKO6DiDQXgg,Mi Familia Mexican Food,"25155 N 67th Ave, Ste 138",Phoenix,AZ,85083.000000,33.713598,-112.202300,3.000000,153,Mexican,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0,POINT (-112.2023003 33.7135985),4013611400,Maricopa County,6114.000000,6166,102950.000000
6056,CIdGsPSUgrLBzSaI72qisQ,Pizza Hut,3145 E Chandler Blvd,Phoenix,AZ,85048.000000,33.304819,-112.016059,3.000000,13,"Italian, Pizza, Chicken Wings",1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,POINT (-112.0160588 33.3048185),4013116714,Maricopa County,1167.140000,3745,106302.000000


In [59]:
#stripping trailing white space from the header
#yelp_compiled.rename(columns=lambda x: x.strip(), inplace=True)
#renaming 
yelp_compiled_cat = yelp_compiled_cat.rename(columns=lambda x: x.strip())

In [60]:
#add up same named columns data
yelp_compiled_cat =yelp_compiled_cat.groupby(yelp_compiled_cat.columns, axis=1).sum(numeric_only=True)

In [61]:
yelp_compiled_cat.shape

(6058, 359)

In [62]:
list(yelp_compiled_cat.columns)

['Acai Bowls',
 'Active Life',
 'Adult Entertainment',
 'Afghan',
 'African',
 'Airport Lounges',
 'Airports',
 'American',
 'Amusement Parks',
 'Antiques',
 'Arabian',
 'Arcades',
 'Archery',
 'Argentine',
 'Armenian',
 'Art Classes',
 'Art Galleries',
 'Arts & Entertainment',
 'Asian Fusion',
 'Audio/Visual Equipment Rental',
 'Auto Repair',
 'Automotive',
 'Baby Gear & Furniture',
 'Bagels',
 'Bakeries',
 'Bangladeshi',
 'Barbeque',
 'Barbers',
 'Bars',
 'Beach Bars',
 'Beauty & Spas',
 'Bed & Breakfast',
 'Beer',
 'Beer Bar',
 'Beer Garden',
 'Beer Gardens',
 'Beverage Store',
 'Bingo Halls',
 'Bistros',
 'Books',
 'Bookstores',
 'Bounce House Rentals',
 'Bowling',
 'Brasseries',
 'Brazilian',
 'Breakfast & Brunch',
 'Breweries',
 'Brewpubs',
 'British',
 'Bubble Tea',
 'Buffets',
 'Burgers',
 'Butcher',
 'Cabaret',
 'Cafes',
 'Cafeteria',
 'Cajun/Creole',
 'Cambodian',
 'Candy Stores',
 'Cantonese',
 'Car Wash',
 'Cardiologists',
 'Caribbean',
 'Casinos',
 'Caterers',
 'Cheese Sho

###### yelp_compiled.head()

In [63]:
yelp_compiled_cat.sum(numeric_only =True)

Acai Bowls              12
Active Life             28
Adult Entertainment      8
Afghan                   7
African                 13
                      ... 
Wine & Spirits         124
Wine Bars              110
Wineries                 2
Wraps                   23
Yelp Events              1
Length: 359, dtype: int64

In [64]:
yelp_compiled_cat = yelp_compiled_cat[yelp_compiled_cat.columns[yelp_compiled_cat.sum()>40]]

In [65]:
print(yelp_compiled_cat.shape)
print(list(yelp_compiled_cat.columns))

(6058, 75)
['American', 'Arts & Entertainment', 'Asian Fusion', 'Bagels', 'Bakeries', 'Barbeque', 'Bars', 'Beer', 'Beer Bar', 'Breakfast & Brunch', 'Buffets', 'Burgers', 'Cafes', 'Casinos', 'Caterers', 'Chicken Wings', 'Chinese', 'Cocktail Bars', 'Coffee & Tea', 'Comfort Food', 'Delis', 'Desserts', 'Diners', 'Ethnic Food', 'Event Planning & Services', 'Fast Food', 'Food', 'Food Delivery Services', 'Food Stands', 'Food Trucks', 'French', 'Gastropubs', 'Gluten-Free', 'Greek', 'Grocery', 'Hawaiian', 'Hot Dogs', 'Hotels', 'Hotels & Travel', 'Ice Cream & Frozen Yogurt', 'Indian', 'Italian', 'Japanese', 'Juice Bars & Smoothies', 'Karaoke', 'Korean', 'Latin American', 'Lounges', 'Mediterranean', 'Mexican', 'Middle Eastern', 'Music Venues', 'Nightlife', 'Noodles', 'Pizza', 'Pubs', 'Salad', 'Sandwiches', 'Seafood', 'Shopping', 'Soup', 'Southern', 'Specialty Food', 'Sports Bars', 'Steakhouses', 'Sushi Bars', 'Tacos', 'Tapas/Small Plates', 'Thai', 'Vegan', 'Vegetarian', 'Venues & Event Spaces', '

In [29]:
cat_drop_list =[[ 'Arts & Entertainment', 'Food''Event Planning & Services',  'Hotels', 'Hotels & Travel','Venues & Event Spaces']]
print(list(yelp_compiled_cat.columns))

['American', 'Arts & Entertainment', 'Asian Fusion', 'Bakeries', 'Barbeque', 'Bars', 'Beer', 'Breakfast & Brunch', 'Buffets', 'Burgers', 'Cafes', 'Casinos', 'Caterers', 'Chicken Wings', 'Chinese', 'Cocktail Bars', 'Coffee & Tea', 'Comfort Food', 'Delis', 'Desserts', 'Diners', 'Ethnic Food', 'Event Planning & Services', 'Fast Food', 'Food', 'Food Trucks', 'Gastropubs', 'Gluten-Free', 'Greek', 'Grocery', 'Hawaiian', 'Hot Dogs', 'Hotels', 'Hotels & Travel', 'Italian', 'Japanese', 'Juice Bars & Smoothies', 'Korean', 'Latin American', 'Lounges', 'Mediterranean', 'Mexican', 'Middle Eastern', 'Nightlife', 'Noodles', 'Pizza', 'Pubs', 'Salad', 'Sandwiches', 'Seafood', 'Soup', 'Specialty Food', 'Sports Bars', 'Steakhouses', 'Sushi Bars', 'Tacos', 'Thai', 'Vegan', 'Vegetarian', 'Venues & Event Spaces', 'Vietnamese', 'Wine & Spirits', 'Wine Bars']


In [74]:
yelp_compiled_cat['ethnic_food'] = yelp_compiled_cat[ 'Ethnic Food']+ yelp_compiled_cat['Greek'] +yelp_compiled_cat['Hawaiian'] + yelp_compiled_cat['Korean']
# yelp_compiled_cat['Latin American']+yelp_compiled_cat ['Mediterranean']+ yelp_compiled_cat['Middle Eastern'] +yelp_compiled_cat['Vietnamese']
# yelp_compiled_cat['Ethnic Food'] =yelp_compiled_compiled_cat
# 'Greek', 'Hawaiian', 'Korean', 'Latin American', 'Mediterranean', 'Vietnamese'

In [ ]:
#getting median income range
q3 =yelp_compiled['median_income'].quantile(.75)
q2 =yelp_compiled['median_income'].quantile(.50)
q1 =yelp_compiled['median_income'].quantile(.25)
def income_classifier (x):
    if x >= q3:
        return "0"
    if x <= q1:
        return "1"
    else:
        return "2"
yelp_compiled['income_range'] = yelp_compiled['median_income'].apply(income_classifier)
#yelp_compiled.head()

In [ ]:
yelp_compiled.describe()

In [ ]:
list(yelp_compiled.columns)

In [ ]:
d = {'business_id':'business','price_range': 'avg_pric_range', 'romantic':'avg_romantic','intimate':'avg_intimate', 'touristy':'average_touristy', 
     'hipster':'avg_hipster', 'divey':'avg_divey', 'classy':'avg_classey', 'trendy':'avg_trendy','upscale': 'avg_upscale',
     'casual':'avg_casual'}
yelp_compiled_tract =yelp_compiled.groupby(['census_tract','city'], as_index= False).agg({'review_count':'sum','stars':'mean','business_id':'count','price_range': 'mean', 'romantic':'mean', 
                                                                                 'intimate':'mean', 'touristy':'mean', 'hipster':'mean', 
                                                                                 'divey':'mean','classy':'mean', 'trendy':'mean','upscale': 'mean', 
 yelp_compiled_tract['resturant_density'] = yelp_compiled_tract['pop_2018']/yelp_compiled_tract['business']                                                                                'casual':'mean', 'median_income':'mean', 'pop_2018':'mean'}).rename(columns=d)

In [ ]:
yelp_compiled_tract.head()

In [ ]:
yelp_compiled_tract.describe()

In [ ]:
# distribution of income
plt.figure(figsize=(8,5))
sns.distplot(yelp_compiled_tract.median_income);

In [ ]:
feature_cols_n = yelp_compiled.columns.drop(['business_id', 'name', 'address', 'city', 'state_left', 'postal_code','latitude',
                                           'longitude','geometry', 'geoid', 'county','census_tract', 'median_income', 'income_range',
                                           'new_categories'])

In [ ]:
X.head()

In [ ]:
X= yelp_compiled[feature_cols_n]
y= yelp_compiled['median_income']

In [ ]:
yelp_compiled['median_income'].value_counts()

In [ ]:
list(yelp_compiled.columns)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size =0.2, random_state = 321)

In [ ]:
%time
regr= LinearRegression()
regr.fit(X_train, y_train)
print('Coefficients: ', regr.coef_)
print('Intercept: ', regr.intercept_)

In [ ]:
from sklearn.metrics import r2_score
y_pred =regr.predict(X)
print('Variance score: %.2f' % regr.score(X,y))
print('Mean absolute error: %.2f' % np.mean(np.absolute(y_pred -y)))
print('Residual sum of square (MSE): %.2f' % np.mean(np.absolute(y_pred -y) ** 2))

print('R2-score: %.2f' % r2_score(y_pred, y) )